In [162]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [163]:
#img_L = cv2.imread('data/tsukuba/tsukuba_1.jpg')
#img_R = cv2.imread('data/tsukuba/tsukuba_2.jpg')
#img_L = cv2.imread("data/sample_left.jpg")  # left image
#img_R = cv2.imread("data/sample_right.jpg")  # right image

img_L = cv2.imread("data/IMG_8301.jpg")  # left image
img_R = cv2.imread("data/IMG_8302.jpg")  # right image
# 必要であれば、以下の行のコメントを外して、画像サイズを変更し、より高速に処理できるようにする。
img_L = cv2.resize(img_L, ((int)(img_L.shape[1]/4), (int)(img_L.shape[0]/4)))

h,w,c = img_L.shape
h_,w_,c_ = img_R.shape

# resize img_R to match img_L
if h_ != h or w_ != w: 
    img_R= cv2.resize(img_R, (w, h), interpolation=cv2.INTER_LINEAR)


cv2.namedWindow('OriginalImage', cv2.WINDOW_AUTOSIZE)
cv2.imshow('OriginalImage', np.hstack((img_L, img_R)))
cv2.waitKey(0)

-1

In [164]:
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(img_L, None)
kp2, des2 = sift.detectAndCompute(img_R, None)

img_L_kp = cv2.drawKeypoints(img_L, kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
img_R_kp = cv2.drawKeypoints(img_R, kp2, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

img_L2_kp = np.hstack((img_L_kp, img_R_kp))
cv2.namedWindow('FeaturePoints', cv2.WINDOW_AUTOSIZE)
cv2.imshow('FeaturePoints', img_L2_kp)
cv2.waitKey(0)

-1

In [165]:
bf = cv2.BFMatcher()
matches = bf.match(des1, des2)
matches = sorted(matches, key = lambda x:x.distance)

img_matches = cv2.drawMatches(img_L, kp1, img_R, kp2, matches[:200], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.namedWindow('MatchedPoints', cv2.WINDOW_AUTOSIZE)
cv2.imshow('MatchedPoints', img_matches)
cv2.waitKey(0)

-1

### 最近隣距離比により、良いマッチングペアを選出する

In [166]:
matches_rt = bf.knnMatch(des1, des2, k=2)
good_matches = []
for m,n in matches_rt:
    if m.distance < 0.75*n.distance:
        good_matches.append(m)

good_matches = sorted(good_matches, key = lambda x:x.distance)
img_good_matches = cv2.drawMatches(img_L, kp1, img_R, kp2, good_matches[:200], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.namedWindow('MatchedPoints200', cv2.WINDOW_AUTOSIZE)
cv2.imshow('MatchedPoints200', img_good_matches)
cv2.waitKey(0)

-1

### 最近距離比から選出した良いマッチングペアにより基礎行列Fを計算する。算出したFにより外れポイントを外す

In [167]:
pts_L = []
pts_R = []

for i,m in enumerate(good_matches):
    pts_L.append(kp1[m.queryIdx].pt)
    pts_R.append(kp2[m.trainIdx].pt)

import numpy as np
pts_L = np.int32(pts_L)
pts_R = np.int32(pts_R)

# RANSACを用いて基礎行列を求める、同時に基礎行列にあったインライアを抽出する
F, mask = cv2.findFundamentalMat(pts_L, pts_R, cv2.RANSAC)

# 基礎行列を用いてエピポーラ線を描画する
pts_L_inliers = pts_L[mask.ravel() == 1]
pts_R_inliers = pts_R[mask.ravel() == 1]
good_matches_inliers = [m for i,m in enumerate(good_matches[:200]) if mask[i,0] == 1]
img_good_matches_inliers = cv2.drawMatches(img_L,kp1,img_R,kp2,good_matches_inliers[:200],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.namedWindow('GoodMatchedPoints', cv2.WINDOW_AUTOSIZE)
cv2.imshow('GoodMatchedPoints', img_good_matches_inliers)
cv2.waitKey(0)


-1

### エピポーラ線を描画する。
- エピポーラ線 ax+by+c=0の(a,b,c)は(r[0],r[1],[r[2]])
- エピポーラ線の両端はx=0の時とx=画像幅の時

In [168]:
# エピポーラ線を描画する関数
def drawlines(img1,img2,lines, pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    _,w,ch = img1.shape
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,ch).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [w, -(r[2]+r[0]*w)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1) # draw epilines in img1
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1) # draw epilines in img1
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1) # draw epilines in img2
    return img1,img2

#右画像の特徴点のエピポーラ線は左の画像に描画する
#pts_R_inliers_200 = pts_R_inliers[0:200,:]
lines_L = cv2.computeCorrespondEpilines(pts_R_inliers.reshape(-1,1,2),2,F)
lines_L = lines_L.reshape(-1,3)
img_L_with_epipole_lines,img_R_with_sift_points = drawlines(img_L.copy(),img_R.copy(),lines_L,pts_L,pts_R)


#左画像の特徴点のエピポーラ線は右の画像に描画する
#pts_L_inliers_200 = pts_L_inliers[0:200,:]
lines_R = cv2.computeCorrespondEpilines(pts_L_inliers.reshape(-1,1,2),1,F)
lines_R = lines_R.reshape(-1,3)
img_R_with_epipole_lines,img_L_with_sift_points = drawlines(img_R.copy(),img_L.copy(),lines_R,pts_R,pts_L)
img_with_epipole_lines = np.hstack((img_L_with_epipole_lines,img_R_with_epipole_lines))
cv2.namedWindow('EpipoleLines', cv2.WINDOW_AUTOSIZE)
cv2.imshow('EpipoleLines', img_with_epipole_lines)
cv2.waitKey(0)

-1

### さらに、Fで外れ値を削除した後の200個のペアを選んでにより基礎行列Fを計算して、エピポーラ線を可視化する

In [169]:
# 下記のコードはH1,H2が不安定
#pts_L = pts_L_inliers
#pts_R = pts_R_inliers

# SIFTの良いペア点を使ったら、安定性が向上。この点数を調整してください!!!
matchingNUM = 200  # SIFTの良いペア点を使う
pts_L = pts_L[0:matchingNUM,:]
pts_R = pts_R[0:matchingNUM,:]

F, mask = cv2.findFundamentalMat(pts_L, pts_R, cv2.RANSAC)

pts_L_inliers = pts_L[mask.ravel() == 1]
pts_R_inliers = pts_R[mask.ravel() == 1]

#右画像の特徴点のエピポーラ線は左の画像に描画する
lines_L = cv2.computeCorrespondEpilines(pts_R_inliers.reshape(-1,1,2),2,F)
lines_L = lines_L.reshape(-1,3)
img_L_with_epipole_lines,img_R_with_sift_points = drawlines(img_L.copy(),img_R.copy(),lines_L,pts_L,pts_R)

#左画像の特徴点のエピポーラ線は右の画像に描画する
lines_R = cv2.computeCorrespondEpilines(pts_L_inliers.reshape(-1,1,2),1,F)
lines_R = lines_R.reshape(-1,3)
img_R_with_epipole_lines,img_L_with_sift_points = drawlines(img_R.copy(),img_L.copy(),lines_R,pts_R,pts_L)
img_with_epipole_lines = np.hstack((img_L_with_epipole_lines,img_R_with_epipole_lines))
cv2.namedWindow('GoodEpipoleLines', cv2.WINDOW_AUTOSIZE)
cv2.imshow('GoodEpipoleLines', img_with_epipole_lines)
cv2.waitKey(0)

-1

## F から整列するホモグラフィ変換Ｈ１，Ｈ２を得て、整列後の画像枠を計算する

In [170]:
h, w ,c= img_L.shape
retval, H1, H2 = cv2.stereoRectifyUncalibrated(
    np.float32(pts_L_inliers),
    np.float32(pts_R_inliers),
    F, imgSize=(w, h), threshold=0
)

imgL_rectify = cv2.warpPerspective(img_L, H1, (w, h))
imgR_rectify = cv2.warpPerspective(img_R, H2, (w, h))

cv2.namedWindow('RectifiedImage', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RectifiedImage', np.hstack((imgL_rectify, imgR_rectify)))
cv2.waitKey(0)


-1

## 奥行きマップ推定の前処理で精度向上
- カラーからグレイ画像へ変換
- ノイズ除去
- 局所ヒストグラム均等化
- 露出の正規化
- 明るすぎる領域の検出と補正


In [171]:
def preprocess_stereo_images(left_img, right_img):
    def preprocess(img):
        # グレースケール変換
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # ガウシアンブラーでノイズ除去
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # CLAHE（局所ヒストグラム均等化）でコントラスト強調（clipLimitを低めに）
        clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8, 8))
        clahe_img = clahe.apply(blurred)

        # 露出の正規化（明るさを均一化）
        normalized = cv2.normalize(clahe_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

        # ハイライト抑制：明るすぎる領域を検出して補正
        highlight_mask = normalized > 240
        suppressed = normalized.copy()
        suppressed[highlight_mask] = cv2.medianBlur(normalized, 5)[highlight_mask]

        return suppressed

    # 左右画像に前処理を適用
    left_processed = preprocess(left_img)
    right_processed = preprocess(right_img)

    return left_processed, right_processed


# 前処理を適用
imgL_preprocessed, imgR_preprocessed = preprocess_stereo_images(imgL_rectify, imgR_rectify)
cv2.namedWindow('PreProcessedImage', cv2.WINDOW_AUTOSIZE)
cv2.imshow('PreProcessedImage', np.hstack((imgL_preprocessed, imgR_preprocessed)))
cv2.waitKey(0)

-1

## ステレオマッチングでStereoデプス推定

| パラメータ名         | 説明                                                                 | 調整のポイント・推奨値例                     |
|----------------------|----------------------------------------------------------------------|----------------------------------------------|
| `minDisparity`       | 最小視差。通常は0。                                                  | カメラの配置によっては負の値も可能。         |
| `numDisparities`     | 視差の範囲。16の倍数で指定。                                         | 例：64, 128。対象物の距離に応じて調整。      |
| `blockSize`          | マッチングに使うブロックサイズ（奇数）。                            | 小さいほど詳細、ただしノイズに弱くなる。     |
| `P1`                 | 小さな視差変化へのペナルティ。滑らかさの制御。                      | `8 * n_channels * blockSize^2` が目安。       |
| `P2`                 | 大きな視差変化へのペナルティ。滑らかさの強さを決定。                | `P1` の4倍以上が推奨。                        |
| `disp12MaxDiff`      | 左右の視差マップの差の許容値。                                       | 小さいほど厳密。例：9。                       |
| `uniquenessRatio`    | 一意性のしきい値（%）。                                              | 高いほどノイズ除去に有効。例：10〜15。       |
| `speckleWindowSize`  | スペックル除去のウィンドウサイズ。                                  | 小さなノイズ領域を除去。例：50〜200。        |
| `speckleRange`       | スペックル除去の視差範囲。                                           | 視差の変化がこの範囲内ならノイズとみなす。   |


In [181]:
# 5. SGBMステレオマッチング
min_disp = -4   # 最小視差
max_disp = 12 + 48  # 最大視差
num_disp = max_disp-min_disp  # 16の倍数
block_size = 5 # ブロックサイズ（マッチングウィンドウのサイズ）

stereo = cv2.StereoSGBM_create(
    minDisparity=min_disp, # 最小視差
    numDisparities=num_disp, # 視差の範囲
    blockSize=block_size, # ブロックサイズ
    # P1とP2は調整可能なパラメータ
    # P1は小さな視差のペナルティ、P2は大きな視差のペナルティ
    P1=8 * c * block_size ** 2, 
    P2=64 * c * block_size ** 2,
    disp12MaxDiff=9, # 左右の視差の最大差
    uniquenessRatio=1, # 一意性比
    speckleWindowSize=100, # スペックルウィンドウサイズ
    speckleRange=16 # スペックル範囲
)
disparity = stereo.compute(imgL_preprocessed, imgR_preprocessed).astype(np.float32) / 16.0
disparity = cv2.normalize(disparity, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
# extend disparity to 3 channels for visualization
disparity_c3 = cv2.cvtColor(disparity.astype(np.uint8), cv2.COLOR_GRAY2BGR)
img_output = np.hstack((img_L,disparity_c3, img_R))
cv2.namedWindow('DisparityMap(SGBM)', cv2.WINDOW_AUTOSIZE)
cv2.imshow("DisparityMap(SGBM)", img_output)
cv2.waitKey(0)

# 画像を保存
cv2.imwrite('data/image_disparity.jpg', img_output)


True